# init

In [1]:
from bson.objectid import ObjectId
from bson.json_util import loads 
from bson.json_util import dumps 

import pandas as pd
import numpy as np
import pymongo
import json
from astropy.table import Table

In [2]:
def drop_index(data):
    data=data.reset_index(drop=True)
    return data

In [3]:
import pymongo
#mon=pymongo.Connection('localhost',port=27017)
#mongod
import datetime
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
mass_db = client["mass_db"]
mycol_mass_db = mass_db["sites"]

#db = client['test-database']
#collection = db['test-collection']

In [4]:
post = {"author": "Brettlv",
         "text": "BH_mass_databass",
         "tags": ["mongodb", "python", "pymongo"],
         "date": datetime.datetime.utcnow()}

posts = mass_db.posts
post_id = posts.insert_one(post).inserted_id

In [5]:
collist_mass_db = mass_db.list_collection_names()
# collist = mydb.collection_names()
if "sites" in collist_mass_db:   # 判断 sites 集合是否存在
    print("集合已存在！")
    
collist_posts = mass_db.list_collection_names()    
if "posts" in collist_posts:   # 判断 sites 集合是否存在
    print("集合已存在！")    

集合已存在！
集合已存在！


In [6]:
for coll in mass_db.list_collection_names():
    print(coll)

posts
sites


In [7]:
for db in client.list_databases():
    print(db)

{'name': 'CLAGN', 'sizeOnDisk': 77824.0, 'empty': False}
{'name': 'admin', 'sizeOnDisk': 49152.0, 'empty': False}
{'name': 'local', 'sizeOnDisk': 110592.0, 'empty': False}
{'name': 'mass_db', 'sizeOnDisk': 167936.0, 'empty': False}
{'name': 'test-database', 'sizeOnDisk': 106496.0, 'empty': False}


mycol_mass_db.find().count()

for i in mycol_mass_db.find():
    print(i)

print("mass count is =", mycol_mass_db.count_documents())

# loadsource

## CLAGN 

In [132]:
CLAGN_statistic_2021_liu=pd.read_excel('/Users/brettlv/blog/IHEP_sync/data/data_clagn/CLAGN_with_RA_Dec_Liuhaov2.xlsx',sheet_name='statistic_paper',header=None)
#CLAGN_list_2021_liu=CLAGN_list_2020[1:]
CLAGN_statistic_2021_liu.columns=['Name','z','type1or2','type','ref1','logM','ref2','']
#CLAGN_statistic_2021_liu
#CLAGN_statistic_2021_liu=CLAGN_statistic_2021_liu[CLAGN_statistic_2021_liu['logM']!=' - ']

In [133]:
CLAGN_statistic_2021_liu_select=CLAGN_statistic_2021_liu[['Name','z','logM']]
CLAGN_statistic_2021_liu_select['Name']=CLAGN_statistic_2021_liu_select['Name'].str.replace(' ','').str.strip()
CLAGN_statistic_2021_liu_select['Note']='Liuetal2021'

<ipython-input-133-a422c5107e64>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CLAGN_statistic_2021_liu_select['Name']=CLAGN_statistic_2021_liu_select['Name'].str.replace(' ','').str.strip()
<ipython-input-133-a422c5107e64>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CLAGN_statistic_2021_liu_select['Note']='Liuetal2021'


In [134]:
CLAGN_statistic_2021_liu_select.columns

Index(['Name', 'z', 'logM', 'Note'], dtype='object')

CLAGN_statistic_2021_liu_select

zarten_CLAGN_statistic = json.loads(CLAGN_statistic_2021_liu_select.T.to_json()).values()
mycol_mass_db.insert_many(zarten_CLAGN_statistic)

In [21]:
print("mycol_mass_db count is = ", mycol_mass_db.count())

mycol_mass_db count is =  0


<ipython-input-21-045be7178bcc>:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  print("mycol_mass_db count is = ", mycol_mass_db.count())


In [135]:
nomass_CLAGN=CLAGN_statistic_2021_liu_select[CLAGN_statistic_2021_liu_select['logM']==' - ']
nomass_CLAGN=drop_index(nomass_CLAGN)

## nomassCLAGN

In [24]:
for name in nomass_CLAGN['Name']:
    print(name)

3C390.3
Mrk372
Mrk609
Mrk728
Mrk883
Mrk993
Mrk1494
NGC1346
NGC2622
NGC4939
NGC5683
WISEAJ004339.35+134436.3
2MASXJ08284278+5444333
2MASSJ09270231+0433080
2MASXJ11180325+4506463
WISEAJ112637.73+513423.0
SDSSJ125258.72+591832.7
SDSSJ130716.99+450645.3
SDSSJ131737.93+102427.7
2MASSJ14345529+5723449
SDSSJ154953.60+112148.3
2MASXJ16050519+4526348
SDSSJ162752.18+541912.5
SDSSJ171353.85+273626.8
2MASXJ20075129-1108346


In [97]:
from astropy.io.votable import parse
from astropy.io.votable import parse_single_table


nomass_CLAGN_votable = parse_single_table("/Users/brettlv/Downloads/CLAGN_statistic_plot/CLAGN/nomass_CLAGN_velocity.xml")

#nomass_CLAGN_votable = parse("/Users/brettlv/Downloads/CLAGN_statistic_plot/CLAGN/nomass_CLAGN_velocity.xml")
#nomass_CLAGN_velocity=Table.read('')
#nomass_CLAGN_velocity

In [99]:
data_nomass_CLAGN_votable = nomass_CLAGN_votable.array

In [104]:
data_nomass_CLAGN_votable

masked_array(data=[(b'2MASXJ15495361+1121481', [237.47343, 11.36339], b'fG.cgi?n=a115&o=2MASXJ15495361+1121481', b'fG.cgi?n=a102&o=2MASXJ15495361+1121481', b'fG.cgi?n=a110&o=2MASXJ15495361+1121481', b'fG.cgi?n=a007&o=2MASXJ15495361+1121481', b'fG.cgi?n=a107&z=g,d,r&o=2MASXJ15495361+1121481', b'fG.cgi?n=a106&o=2MASXJ15495361+1121481', b'fG.cgi?n=a103&o=2MASXJ15495361+1121481', b'fG.cgi?n=a001&o=2MASXJ15495361+1121481', b'fG.cgi?n=a002&o=2MASXJ15495361+1121481', b'fG.cgi?n=a108&o=2MASXJ15495361+1121481', b'fG.cgi?n=a101&o=2MASXJ15495361+1121481', b'fG.cgi?n=a113&o=2MASXJ15495361+1121481', b'fG.cgi?n=a004&o=2MASXJ15495361+1121481', b'fG.cgi?n=a005&o=2MASXJ15495361+1121481', b'fG.cgi?n=a006&o=2MASXJ15495361+1121481', b'fG.cgi?n=a112&o=2MASXJ15495361+1121481', b'fG.cgi?n=a003&z=g,d,r&o=2MASXJ15495361+1121481', b'fG.cgi?n=a009&z=g,d,r&o=2MASXJ15495361+1121481', b'fG.cgi?n=a010&z=g,d,r&o=2MASXJ15495361+1121481', b'fG.cgi?n=a011&o=2MASXJ15495361+1121481', b'fG.cgi?n=a104&o=2MASXJ15495361+11214

In [101]:
data_nomass_CLAGN_votable['objname']

masked_array(data=[b'2MASXJ15495361+1121481', b'2MASXJ16275221+5419126',
                   b'IC1854', b'NGC1346', b'NGC2622', b'NGC5683',
                   b'PGC012801', b'PGC033214', b'PGC053662', b'PGC057026',
                   b'PGC058359', b'PGC139560', b'PGC165838',
                   b'PGC2588950', b'SDSSJ130716.99+450645.2',
                   b'SDSSJ131737.93+102427.7', b'SDSSJ171353.85+273626.8',
                   b'UGC00987'],
             mask=[False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False],
       fill_value=b'N/A',
            dtype='|S30')

In [116]:
for i in nomass_CLAGN['Name']:
    if i in BAT_pd['Name'].values:
        print(i)

3C390.3
Mrk728
NGC4939
NGC5683


In [117]:
for i in nomass_CLAGN['Name']:
    if i in BAT_lit['Name'].values:
        print(i)

3C390.3
Mrk728
NGC4939
NGC5683


## Check CLAGN Name

In [140]:
for i in CLAGN_statistic_2021_liu_select['Name']:
    if i in BAT_pd['Name'].values:
        print(i,'BAT')
        
    elif i in LLAGN['Name'].values:
        print(i,'LLAGN')
        
    
    elif i in BAT_corona_parameter_select['Name'].values:
        print(i,'BAT_cor')
    

1H0419-577 BAT
3C390.3 BAT
Fairall9 BAT
HE1136-2304 BAT
Mrk6 BAT
Mrk590 BAT
Mrk728 BAT
Mrk926 BAT
Mrk1018 BAT
NGC1365 BAT
NGC1566 BAT
NGC2992 BAT
NGC3516 BAT
NGC4051 BAT
NGC4151 BAT
NGC4388 BAT
NGC4395 BAT
NGC4507 BAT
NGC4939 BAT
NGC5548 BAT
NGC5683 BAT
NGC6300 BAT
NGC7469 BAT
NGC7582 BAT


## myquery_nomassclagn

In [90]:
for name in nomass_CLAGN['Name']:
    myquery_nomassclagn= {"Name":name.strip()} 
    for x in mycol_mass_db.find(myquery_nomassclagn):
        if x['logM']!=' - ':
            print(x)

{'_id': ObjectId('61457ff4003f629b7cd39d99'), 'Name': 'NGC4939', 'logM': 7.52, 'Type': 2.0, 'Note': 'BAT_I', 'reference': '2017ApJ...850...74K'}


In [114]:
np.log10(2.00E+08),np.log10(2.00E+09)

(8.301029995663981, 9.301029995663981)

## delete all 

x = mycol_mass_db.delete_many({})#删除所有
print(x.deleted_count, "个文档已删除")

## delete blank mass

myquery_0 = { 'logM': ' - '}#删除所有 'logM' 字段中' - '的文档:
x = mycol_mass_db.delete_many(myquery_0)
print(x.deleted_count, "个文档已删除")

In [22]:
for x in mycol_mass_db.find({'Note': 'Liuetal2021'})[:5]:
    print(x)

{'_id': ObjectId('6143279bb96dceca98da608a'), 'Name': '1ES 1927+654', 'z': 0.017, 'logM': 7.3, 'Note': 'Liuetal2021', 'Type': 'CLAGN'}
{'_id': ObjectId('6143279bb96dceca98da608b'), 'Name': '1H 0419-577', 'z': 0.104, 'logM': 8.58, 'Note': 'Liuetal2021', 'Type': 'CLAGN'}
{'_id': ObjectId('6143279bb96dceca98da608d'), 'Name': 'ESO 362-G18', 'z': 0.012, 'logM': 7.65, 'Note': 'Liuetal2021', 'Type': 'CLAGN'}
{'_id': ObjectId('6143279bb96dceca98da608e'), 'Name': 'Fairall 9', 'z': 0.046, 'logM': 8.41, 'Note': 'Liuetal2021', 'Type': 'CLAGN'}
{'_id': ObjectId('6143279bb96dceca98da608f'), 'Name': 'HE 1136-2304', 'z': 0.027, 'logM': 7.58, 'Note': 'Liuetal2021', 'Type': 'CLAGN'}


In [45]:
myquery_1 = {'Note': 'Liuetal2021'}
newvalues_1 = { "$set": { "Type": "CLAGN" } }
mycol_mass_db.update_many(myquery_1, newvalues_1)

## count CLAGN

In [47]:
print("mycol_mass_db count is = ", mycol_mass_db.count({"Type": "CLAGN" }))

mycol_mass_db count is =  116


<ipython-input-47-22a4998150ef>:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  print("mycol_mass_db count is = ", mycol_mass_db.count({"Type": "CLAGN" }))


## LLAGN

In [136]:
LLAGN=pd.read_csv('/Users/brettlv/Downloads/CLAGN_statistic_plot/Source_lists/LLAGNtable.txt',delimiter='\t',header=0)
#CLAGN_list_2021_liu=CLAGN_list_2020[1:]
LLAGN.columns=['Name', 'lg(L2-10keV)', 'Sigma_dis', 'logM', 'lg(LEdd)',
       'lg(Lbol)', 'lg(Lbol/LEdd)', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9']

LLAGN['reference']='2009ApJ...699..626H'
LLAGN_select=LLAGN[['Name','logM','reference']]
print(LLAGN_select.columns)

Index(['Name', 'logM', 'reference'], dtype='object')


#LLAGN['Name'].to_csv('/Users/brettlv/Downloads/CLAGN_statistic_plot/Source_lists/LLAGN_Name.txt',index=False)

zarten_LLAGN = json.loads(LLAGN_select.T.to_json()).values()
mycol_mass_db.insert_many(zarten_LLAGN)

myquery_LLAGN = {'reference': '2009ApJ...699..626H'}
newvalues_LLAGN = { "$set": { "Type": "LLAGN" } }
mycol_mass_db.update_many(myquery_LLAGN, newvalues_LLAGN)

myquery_LLAGN = {"reference": "2009ApJ...699..626H"} #LLAGN
for x in mycol_mass_db.find(myquery_LLAGN)[:5]:
    print(x)

print("LLAGN count is = ", mycol_mass_db.count({"reference": "2009ApJ...699..626H" }))

In [118]:
LLAGN_gu2009=pd.read_excel('/Users/brettlv/blog/IHEP_sync/data/AGNmass/LLAGN_QSO/LLAGN_Gu2009.xlsx')

In [128]:
LLAGN_gu2009.columns

Index(['Name', 'z', 'Sat', '(2–10 keV)', 'log L2−10keV', 'References',
       '∗ log MBH', 'References.1', 'Type', 'logM'],
      dtype='object')

In [131]:
LLAGN_gu2009[['Name','∗ log MBH','Type']]

,Name,∗ log MBH,Type
0,NGC 1275,8.51,Seyfert galaxies
1,NGC 2639,8.02,Seyfert galaxies
2,NGC 2655,7.77,Seyfert galaxies
3,NGC 2685,7.15,Seyfert galaxies
4,NGC 3031,7.80b,Seyfert galaxies
5,NGC 3147,8.79,Seyfert galaxies
6,NGC 3227,7.59c,Seyfert galaxies
7,NGC 3486,6.14,Seyfert galaxies
8,NGC 3516,7.36c,Seyfert galaxies
9,NGC 3941,8.15,Seyfert galaxies


In [126]:
for index,i in enumerate(LLAGN_gu2009['∗ log MBH']):
    if type(i)==float:
        LLAGN_gu2009.loc[index,'logM']=i  
        
    elif i.find('b')>0:
        LLAGN_gu2009.loc[index,'logM']=i.split('b')[0]
            
    elif i.find('c')>0:
        LLAGN_gu2009.loc[index,'logM']=i.split('c')[0]
        
    elif i.find('d')>0:
        LLAGN_gu2009.loc[index,'logM']=i.split('d')[0]        
    
    elif i.find('e')>0:
        LLAGN_gu2009.loc[index,'logM']=i.split('e')[0]        
        
    elif i.find('f')>0:
        LLAGN_gu2009.loc[index,'logM']=i.split('f')[0] 


AttributeError: 'int' object has no attribute 'find'

## count LLAGN

In [23]:
print("LLAGN mass count is =", mycol_mass_db.count_documents({"reference": "2009ApJ...699..626H" }))

LLAGN mass count is = 0


In [26]:
x=mycol_mass_db.delete_many({"reference": "2009ApJ...699..626H" })
x.deleted_count

0

## BAT source

In [26]:
BATname_1=Table.read('/Users/brettlv/blog/IHEP_sync/data/AGNmass/Swift_BAT_AGN_Spectroscopic_Survey_I/table2.dat',
                      readme="/Users/brettlv/blog/IHEP_sync/data/AGNmass/Swift_BAT_AGN_Spectroscopic_Survey_I/ReadMe",
                      format="ascii.cds",)

BATmass_1=Table.read('/Users/brettlv/blog/IHEP_sync/data/AGNmass/Swift_BAT_AGN_Spectroscopic_Survey_I/table4.dat',
                      readme="/Users/brettlv/blog/IHEP_sync/data/AGNmass/Swift_BAT_AGN_Spectroscopic_Survey_I/ReadMe",
                      format="ascii.cds",)

### CNAME logM

In [32]:
BAT_pd=pd.DataFrame([])
BAT_pd['Name']=BATname_1['CName']
BAT_pd['Name']=BAT_pd['Name'].str.replace(' ','')
BAT_pd['Name']=BAT_pd['Name'].str.strip()
BAT_pd['logM']=BATmass_1['logMbh']
BAT_pd['Type']=BATname_1['Type']
BAT_pd['Note']='BAT_I'
BAT_pd['reference']='2017ApJ...850...74K'

In [37]:
BAT_pd_select=BAT_pd[BAT_pd['logM']>0]
print(len(BAT_pd_select))

BAT_pd_noselect=BAT_pd[BAT_pd['logM']==0]
print(len(BAT_pd_noselect))

201
440


In [29]:
BAT_pd.columns

Index(['Name', 'logM', 'Type', 'Note', 'reference'], dtype='object')

In [38]:
zarten_BAT_I = json.loads(BAT_pd_select.T.to_json()).values()
mycol_mass_db.insert_many(zarten_BAT_I)

### logM literature

In [39]:
BAT_lit=pd.DataFrame([])
BAT_lit['Name']=BATname_1['CName']
BAT_lit['Name']=BAT_lit['Name'].str.replace(' ','')
BAT_lit['Name']=BAT_lit['Name'].str.strip()

BAT_lit['logM']=BATmass_1['logMbhlit']
BAT_lit['Type']=BATname_1['Type']
BAT_lit['Note']='BAT_I_lit'
BAT_lit['reference']='2017ApJ...850...74K'

In [40]:
BAT_lit_select=BAT_lit[BAT_lit['logM']>0]
BAT_lit_select=drop_index(BAT_lit_select)

In [43]:
len(BAT_pd),len(BAT_lit),len(BAT_pd_select),len(BAT_lit_select)

(641, 641, 201, 15)

In [41]:
zarten_BAT_lit_select = json.loads(BAT_lit_select.T.to_json()).values()
mycol_mass_db.insert_many(zarten_BAT_lit_select)

myquery_BAT_0 = { 'logM': 0.0}#删除所有 'logM' 字段中' - '的文档:
x = mycol_mass_db.delete_many(myquery_BAT_0)
print(x.deleted_count, "个文档已删除")

In [8]:
print("BAT_I count is =", mycol_mass_db.find({"reference": "2017ApJ...850...74K"}).count())

BAT_I count is = 216


<ipython-input-8-5f0e71df3e8b>:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  print("BAT_I count is =", mycol_mass_db.find({"reference": "2017ApJ...850...74K"}).count())


In [9]:
myquery_BAT_I= {"reference": "2017ApJ...850...74K"} #BAT_I
for x in mycol_mass_db.find(myquery_BAT_I):
    print(x)

{'_id': ObjectId('61457ff4003f629b7cd39d14'), 'Name': '2MASXJ00004876-0709117', 'logM': 7.97, 'Type': 1.9, 'Note': 'BAT_I', 'reference': '2017ApJ...850...74K'}
{'_id': ObjectId('61457ff4003f629b7cd39d15'), 'Name': '2MASXJ00032742+2739173', 'logM': 7.85, 'Type': 2.0, 'Note': 'BAT_I', 'reference': '2017ApJ...850...74K'}
{'_id': ObjectId('61457ff4003f629b7cd39d16'), 'Name': '2MASXJ00091156-0036551', 'logM': 8.91, 'Type': 2.0, 'Note': 'BAT_I', 'reference': '2017ApJ...850...74K'}
{'_id': ObjectId('61457ff4003f629b7cd39d17'), 'Name': '2MASXJ00210753-1910056', 'logM': 9.16, 'Type': 1.9, 'Note': 'BAT_I', 'reference': '2017ApJ...850...74K'}
{'_id': ObjectId('61457ff4003f629b7cd39d18'), 'Name': 'NGC235A', 'logM': 8.6, 'Type': 1.9, 'Note': 'BAT_I', 'reference': '2017ApJ...850...74K'}
{'_id': ObjectId('61457ff4003f629b7cd39d19'), 'Name': 'Mrk348', 'logM': 7.61, 'Type': 1.9, 'Note': 'BAT_I', 'reference': '2017ApJ...850...74K'}
{'_id': ObjectId('61457ff4003f629b7cd39d1a'), 'Name': 'ESO195-IG021NED03

## BAT_corona_parameter

In [10]:
BAT_corona_parameter=pd.read_excel('/Users/brettlv/blog/IHEP_sync/data/AGNmass/2021MNRAS.506.4960H_table4.xlsx',header=0)
BAT_corona_parameter=BAT_corona_parameter[BAT_corona_parameter['Object'].notnull()]
BAT_corona_parameter=drop_index(BAT_corona_parameter)

In [11]:
BAT_corona_parameter.columns

Index(['Object', 'Type', 'Redshift', 'log(MBH)', 'log(λe)', 'nH', 'Unnamed: 6',
       'Efold', 'R', 'Unnamed: 9', 'χ2/d.o.f.', 'Ref.', '2021MNRAS.506.4960H'],
      dtype='object')

In [60]:
for index in range(len(BAT_corona_parameter['log(MBH)'])):
    if BAT_corona_parameter.loc[index,'log(MBH)'].find('+')>0:
        logMbh=float(BAT_corona_parameter.loc[index,'log(MBH)'].split('+')[0])
    elif BAT_corona_parameter.loc[index,'log(MBH)'].find('±')>0:
        logMbh=float(BAT_corona_parameter.loc[index,'log(MBH)'].split('±')[0])
           
    BAT_corona_parameter.loc[index,'logM']=logMbh    
    BAT_corona_parameter.loc[index,'Name']=BAT_corona_parameter.loc[index,'Object'].replace(' ','').strip()


In [61]:
BAT_corona_parameter_select=BAT_corona_parameter[['Name','Type','Redshift','logM']]
#BAT_corona_parameter_select=BAT_corona_parameter_select['Name'].str.replace(' ','')
#BAT_corona_parameter_select=BAT_corona_parameter_select['Name'].strip()
BAT_corona_parameter_select['reference']='2021MNRAS.506.4960H'
BAT_corona_parameter_select['Note']='BAT_corona'

<ipython-input-61-8557aaedc80e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BAT_corona_parameter_select['reference']='2021MNRAS.506.4960H'
<ipython-input-61-8557aaedc80e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BAT_corona_parameter_select['Note']='BAT_corona'


In [62]:
BAT_corona_parameter_select.columns

Index(['Name', 'Type', 'Redshift', 'logM', 'reference', 'Note'], dtype='object')

zarten_BAT_corona = json.loads(BAT_corona_parameter_select.T.to_json()).values()
mycol_mass_db.insert_many(zarten_BAT_corona)

In [91]:
BAT_corona_parameter_select['Name']

0               Mrk1501
1               Mrk1148
2              Fairall9
3                Mrk359
4                NGC931
5               NGC1052
6               NGC1068
7                 3C109
8                 3C120
9                Ark120
10            ESO362−18
11                 Mrk3
12       IRAS09149−6206
13              NGC3227
14              NGC3998
15              NGC4151
16               Mrk766
17                3C273
18              NGC4579
19              NGC4593
20              NGC4785
21               Mrk273
22               Mrk841
23              Mrk1392
24                3C382
25    SwiftJ2127.4+5654
26              IIZw171
27              NGC7314
28               Mrk915
29           MR2251−178
30              NGC7469
31               Mrk926
32              NGC7582
Name: Name, dtype: object

# QSO

# EVQ

# count

In [66]:
print("BAT_I mass count is =", mycol_mass_db.count_documents({"Note": "BAT_I"}))

BAT_I mass count is = 201


In [67]:
print("BAT_I_lit mass count is =", mycol_mass_db.count_documents({"Note": "BAT_I_lit"}))

BAT_I_lit mass count is = 15


In [68]:
print("BAT mass count is =", mycol_mass_db.count_documents({"reference": "2017ApJ...850...74K"}))

BAT mass count is = 216


In [69]:
print("CLAGN mass count is =", mycol_mass_db.count_documents({"Note": "Liuetal2021"}))

CLAGN mass count is = 141


In [70]:
print("LLAGN mass count is =", mycol_mass_db.count_documents({"Type": "LLAGN"}))

LLAGN mass count is = 0


In [74]:
print("CLAGN mass >8 count is =", mycol_mass_db.count_documents({"Note": "Liuetal2021","logM": {"$gt":8}}))

LLAGN mass count is = 43


In [76]:
print("CLAGN mass <8 count is =", mycol_mass_db.count_documents({"Note": "Liuetal2021","logM": {"$lt":8}}))

CLAGN mass <8 count is = 62


In [79]:
print("CLAGN no mass count is =", mycol_mass_db.count_documents({"Note": "Liuetal2021","logM": " - "}))
                                                                  
                                                                  

CLAGN no mass count is = 25


In [80]:
print("BAT_corona_parameter mass count is =", mycol_mass_db.count_documents({"reference": "2021MNRAS.506.4960H"}))

BAT_corona_parameter mass count is = 33


mycol_mass_db.find({"logM": {"$gt": 9}}).count()

mycol_mass_db.find({"logM": {"$lt": 9}}).count()

mycol_mass_db.find({"logM": {"$eq": 9}}).count()

In [12]:
import pprint
for x in mycol_mass_db.find({"logM": {"$gt": 10}}).sort("Name"):
    pprint.pprint(x)

In [13]:
import pprint
for x in mycol_mass_db.find({"logM": {"$lt": 10}}).sort("Name"):
    pprint.pprint(x)

{'Name': '1ES1426+428',
 'Note': 'BAT_I',
 'Type': 0.0,
 '_id': ObjectId('61457ff4003f629b7cd39daf'),
 'logM': 8.87,
 'reference': '2017ApJ...850...74K'}
{'Name': '1ES1927+654',
 'Note': 'Liuetal2021',
 '_id': ObjectId('61457b4b003f629b7cd39c87'),
 'logM': 7.3,
 'z': 0.017}
{'Name': '1H0419-577',
 'Note': 'Liuetal2021',
 '_id': ObjectId('61457b4b003f629b7cd39c88'),
 'logM': 8.58,
 'z': 0.104}
{'Name': '2MASSJ01264811-0839481',
 'Note': 'Liuetal2021',
 '_id': ObjectId('61457b4b003f629b7cd39cb7'),
 'logM': 8.08,
 'z': 0.198}
{'Name': '2MASSJ07451198+3809114',
 'Note': 'Liuetal2021',
 '_id': ObjectId('61457b4b003f629b7cd39cbe'),
 'logM': 9.1,
 'z': 0.236}
{'Name': '2MASSJ10022021+4509276',
 'Note': 'Liuetal2021',
 '_id': ObjectId('61457b4b003f629b7cd39cd0'),
 'logM': 8.7,
 'z': 0.401}
{'Name': '2MASSJ11132967+5313393',
 'Note': 'Liuetal2021',
 '_id': ObjectId('61457b4b003f629b7cd39cda'),
 'logM': 7.8,
 'z': 0.239}
{'Name': '2MASSJ12335916+0842112',
 'Note': 'Liuetal2021',
 '_id': ObjectId

{'Name': 'MCG+06-16-028',
 'Note': 'BAT_I',
 'Type': 1.9,
 '_id': ObjectId('61457ff4003f629b7cd39d4b'),
 'logM': 7.34,
 'reference': '2017ApJ...850...74K'}
{'Name': 'MCG+06-24-008',
 'Note': 'BAT_I',
 'Type': 2.0,
 '_id': ObjectId('61457ff4003f629b7cd39d69'),
 'logM': 7.17,
 'reference': '2017ApJ...850...74K'}
{'Name': 'MCG+08-03-018',
 'Note': 'BAT_I',
 'Type': 2.0,
 '_id': ObjectId('61457ff4003f629b7cd39d20'),
 'logM': 8.41,
 'reference': '2017ApJ...850...74K'}
{'Name': 'MCG+09-19-015NED02',
 'Note': 'BAT_I',
 'Type': 2.0,
 '_id': ObjectId('61457ff4003f629b7cd39d72'),
 'logM': 8.58,
 'reference': '2017ApJ...850...74K'}
{'Name': 'MCG+10-14-025',
 'Note': 'BAT_I',
 'Type': 1.9,
 '_id': ObjectId('61457ff4003f629b7cd39d60'),
 'logM': 8.35,
 'reference': '2017ApJ...850...74K'}
{'Name': 'MCG+10-17-061',
 'Note': 'BAT_I',
 'Type': 2.0,
 '_id': ObjectId('61457ff4003f629b7cd39d7b'),
 'logM': 8.61,
 'reference': '2017ApJ...850...74K'}
{'Name': 'MCG+11-11-032',
 'Note': 'BAT_I',
 'Type': 2.0,
 

 'logM': 8.87,
 'reference': '2017ApJ...850...74K'}
{'Name': 'NGC5290',
 'Note': 'BAT_I',
 'Type': 2.0,
 '_id': ObjectId('61457ff4003f629b7cd39da7'),
 'logM': 7.76,
 'reference': '2017ApJ...850...74K'}
{'Name': 'NGC5548',
 'Note': 'Liuetal2021',
 '_id': ObjectId('61457b4b003f629b7cd39caa'),
 'logM': 7.51,
 'z': 0.017}
{'Name': 'NGC5610',
 'Note': 'BAT_I',
 'Type': 2.0,
 '_id': ObjectId('61457ff4003f629b7cd39dae'),
 'logM': 7.81,
 'reference': '2017ApJ...850...74K'}
{'Name': 'NGC5674',
 'Note': 'BAT_I',
 'Type': 2.0,
 '_id': ObjectId('61457ff4003f629b7cd39db0'),
 'logM': 7.65,
 'reference': '2017ApJ...850...74K'}
{'Name': 'NGC5728',
 'Note': 'BAT_I',
 'Type': 1.9,
 '_id': ObjectId('61457ff4003f629b7cd39db2'),
 'logM': 8.07,
 'reference': '2017ApJ...850...74K'}
{'Name': 'NGC5899',
 'Note': 'BAT_I',
 'Type': 2.0,
 '_id': ObjectId('61457ff4003f629b7cd39db6'),
 'logM': 8.66,
 'reference': '2017ApJ...850...74K'}
{'Name': 'NGC5995',
 'Note': 'BAT_I',
 'Type': 1.9,
 '_id': ObjectId('61457ff400

In [85]:
for x in mycol_mass_db.find().sort("Name"):
    print(x['Name'],x['logM'],x['Note'])

1ES1426+428 8.87 BAT_I
1ES1927+654 7.3 Liuetal2021
1H0419-577 8.58 Liuetal2021
2MASSJ01264811-0839481 8.08 Liuetal2021
2MASSJ07451198+3809114 9.1 Liuetal2021
2MASSJ09270231+0433080  -  Liuetal2021
2MASSJ10022021+4509276 8.7 Liuetal2021
2MASSJ11132967+5313393 7.8 Liuetal2021
2MASSJ12335916+0842112 8.7 Liuetal2021
2MASSJ14284671+1723530 7.6 Liuetal2021
2MASSJ14345529+5723449  -  Liuetal2021
2MASSJ15361278+0342457 8.2 Liuetal2021
2MASSJ15373408+4613585 8 Liuetal2021
2MASSJ16011122+4745095 7.9 Liuetal2021
2MASSJ16171142+0638333 8 Liuetal2021
2MASSJ21020044+0005020 8 Liuetal2021
2MASSJ22053771-0711147 8 Liuetal2021
2MASXJ00004876-0709117 7.97 BAT_I
2MASXJ00032742+2739173 7.85 BAT_I
2MASXJ00091156-0036551 8.91 BAT_I
2MASXJ00210753-1910056 9.16 BAT_I
2MASXJ01073963-1139117 8.08 BAT_I
2MASXJ01392400+2924067 8.08 BAT_I
2MASXJ02420381+0510061 9.23 BAT_I
2MASXJ02485937+2630391 9.15 BAT_I
2MASXJ03252346-5635443 9.1 BAT_I
2MASXJ03534246+3714077 8.54 BAT_I
2MASXJ04332716-5843346 8.5 BAT_I
2MASXJ0444

In [15]:
print("mass count is =", mycol_mass_db.count_documents({"Name": "NGC4939"}))

mass count is = 2


# find

In [16]:
for x in mycol_mass_db.find({"Name": "NGC4939"}):
    print(x)

{'_id': ObjectId('61457b4b003f629b7cd39ca9'), 'Name': 'NGC4939', 'z': 0.01, 'logM': ' - ', 'Note': 'Liuetal2021'}
{'_id': ObjectId('61457ff4003f629b7cd39d99'), 'Name': 'NGC4939', 'logM': 7.52, 'Type': 2.0, 'Note': 'BAT_I', 'reference': '2017ApJ...850...74K'}
